In [1]:
import numpy as np
from qibo.models import Circuit, Grover
from qibo import gates
from scipy.special import binom as binomial

[Qibo 0.1.6.dev2|INFO|2021-09-21 18:22:56]: Using qibojit backend on /CPU:0


In [2]:
from functions_new import lin_sys, isd_quantum

In [3]:
r=3
n=5

test_isd = lin_sys(n, r, lee_brickell=False)

test_isd.print_instance()

Solve all possible linear systems, where

H =
[[0 1 1 0 0]
 [1 1 0 1 0]
 [0 1 0 0 1]] 

b =
[[0]
 [1]
 [0]] 

Solutions:

Column choice: 11100     Syndrome: 010
Column choice: 11001     Syndrome: 010
Column choice: 10101     Syndrome: 100
Column choice: 01110     Syndrome: 001
Column choice: 01101     Syndrome: 111
Column choice: 01011     Syndrome: 010
Column choice: 00111     Syndrome: 010




In [4]:
test = isd_quantum(test_isd.H, test_isd.s, lee_brickell=False)

In [5]:
test.check_isd(solutions=test_isd.solutions)

-------------------------------------------------------
| Column choices  | Syndrome        | Probability     |
-------------------------------------------------------
| 01110           | 001             | 0.1014          |
-------------------------------------------------------
| 00111           | 010             | 0.1001          |
-------------------------------------------------------
| 01011           | 010             | 0.0995          |
-------------------------------------------------------
| 11001           | 010             | 0.1033          |
-------------------------------------------------------
| 11010           | 010             | 0.0989          |
-------------------------------------------------------
| 11100           | 010             | 0.1007          |
-------------------------------------------------------
| 10011           | 100             | 0.0947          |
-------------------------------------------------------
| 10101           | 100             | 0.1011    

In [6]:
target = 3

In [7]:
oracle = test.isd_oracle(target)
superposition = test.superposition_circuit()
sup_size = int(binomial(n, r))

In [8]:
check = test.check_solution
check_args = (test_isd.H, test_isd.s, target)

In [9]:
#isd_grover = Grover(oracle, superposition_circuit=superposition, superposition_qubits=n,
#                    superposition_size=sup_size, check=check, check_args=check_args, iterative=True)

isd_grover = Grover(oracle, superposition_circuit=superposition, superposition_qubits=n,
                            superposition_size=sup_size, number_solutions=1,
                            check=check, check_args=check_args)



In [10]:
solution, iterations = isd_grover()
print(f'Solution found: {solution} in {iterations} Grover iterations.\n')

Solution found: ['01101'] in 2 Grover iterations.



In [12]:
test.check_solution('01101', test_isd.H, test_isd.s, target, lee_brickell=0)

True

In [14]:
test.check_solution('00111', test_isd.H, test_isd.s, target, lee_brickell=0)

False

In [15]:
isd_grover = Grover(oracle, superposition_circuit=superposition, superposition_qubits=n,
                    superposition_size=sup_size, check=check, check_args=check_args, iterative=True)



In [16]:
solution, iterations = isd_grover()
print(f'Solution found: {solution} in {iterations} Grover iterations.\n')

Solution found: 01101 in 2 Grover iterations.



In [67]:
r=3
n=5

test_lb = lin_sys(n, r, lee_brickell=True)


test_lb.print_instance()

Solve all possible linear systems, where

H =
[[0 0 1 0 0]
 [1 1 0 0 1]
 [0 1 0 1 1]] 

b =
[[1]
 [1]
 [0]] 

Solutions:

Column choice: 11100     Syndrome: 101
Column choice: 10110     Syndrome: 011
Column choice: 10101     Syndrome: 011
Column choice: 01110     Syndrome: 111
Column choice: 00111     Syndrome: 111




In [68]:
test = isd_quantum(test_lb.H, test_lb.s, lee_brickell=True)

In [69]:
test.check_isd(solutions=test_lb.solutions)

-------------------------------------------------------
| Column choices  | Syndrome        | Probability     |
-------------------------------------------------------
| 01101           | 011             | 0.103           |
-------------------------------------------------------
| 10101           | 011             | 0.0995          |
-------------------------------------------------------
| 10110           | 011             | 0.1049          |
-------------------------------------------------------
| 10011           | 101             | 0.1023          |
-------------------------------------------------------
| 11010           | 101             | 0.0984          |
-------------------------------------------------------
| 11100           | 101             | 0.0948          |
-------------------------------------------------------
| 11001           | 110             | 0.0973          |
-------------------------------------------------------
| 00111           | 111             | 0.1014    

In [62]:
target = 3
p = 1

In [63]:
oracle = test.lb_oracle(target, p)
superposition = test.superposition_circuit()
sup_size = int(binomial(n, r))

In [64]:
check = test.check_solution
check_args = (test_lb.H, test_lb.s, target, p)

In [65]:
isd_grover = Grover(oracle, superposition_circuit=superposition, superposition_qubits=n,
                    superposition_size=sup_size, check=check, check_args=check_args, iterative=True)

In [66]:
solution, iterations = isd_grover()
print(f'Solution found: {solution} in {iterations} Grover iterations.\n')

Solution found: 0111 in 3 Grover iterations.



In [58]:
test.check_solution('1110', test_lb.H, test_lb.s, target, lee_brickell=p)

False

In [61]:
test.check_solution('1101', test_lb.H, test_lb.s, target, lee_brickell=p)

False

In [69]:
def check_solution(perm, A, b, target, lee_brickell=0):
    '''Check if a given permutation outputs the desired target weight.
    Args:
        perm (list): choice of columns to check.
        H (np.array): parity check matrix.
        s (np.array): original syndrome.
        target (int): target weight of the syndrome by the end.

    Returns:
        (bool): True if the weight of the resulting syndrome is equal to the target.

    '''
    def find_set_bits(x, n):
        bits = []
        for i in range(n):
            mask = 1 << i
            if mask & x:
                bits.append(i)
        return bits

    def solve_gf2(A,t):
        r=A.shape[0]
        b=np.copy(t)
        for i in range(r):    
            for j in range(i+1,r):
                if A[j,i]==1:
                    if A[i,i]!=1:
                        A[i]^=A[j]
                        b[i]^=b[j]
                    A[j]^=A[i]
                    b[j]^=b[i]
            if A[i,i]!=1:
                return False
        for i in reversed(range(r)):
            for j in range(i):
                if A[j,i]==1:
                    A[j]^=A[i]
                    b[j]^=b[i]
        return b

    n = A.shape[1]
    k = A.shape[0]

    P=np.matrix(A).transpose()
    L=[]
    if lee_brickell:
        range_n=[i for i in reversed(range(n))]
    else:
        range_n=[i for i in reversed(range(n-k))]+[i for i in range(n-k,n)]
    for i in range_n:
        if perm[i]=="1":
            L.append(P[i].tolist()[0])
    #for i in range(n):
    #    if perm[i] == "1":
    #        L.append(P[i].tolist()[0])

    if not(lee_brickell):
        Hp=np.matrix(L).transpose()
        x=solve_gf2(Hp,b)%2
        return np.int(np.sum(x))==target
    else:
        for i in range(n):
            if perm[i]=="0":
                L.append(P[i].tolist()[0])
        Hp=np.matrix(L).transpose()
        x=solve_gf2(Hp,b)%2
        try:
            len(x)
        except:
            return False
        p=lee_brickell
        correct=False

        length=n-k
        setb = (1 << p) - 1;
        limit = (1 << length);
        while setb < limit:
            res=[i for i in x]
            #get positions of set bits in setb

            columns_to_add=find_set_bits(setb,k)

            #add those columns to syndrome, note that indices need to be shifted by k
            for i in columns_to_add:
                for j in range(len(res)):
                    res[j]^=Hp[j,k+i]
            if np.int(np.sum(res))==target-p:
                return True

            #gives next binary number with p bits out of k set to one
            c = setb & - setb;
            v = setb + c;
            setb = int(((v ^ setb) >> 2) / c) | int(v);
        return False